# One symbol program
## Run a collection of programs to get naked naked options for a symbol

- [x] Get symbols. Make new if not present or old.
- [x] Get a symbol
- [x] Update its chain strikes
- [] Get strikes closest to unds for each expiry
- [] Get margin and iv for the earliest strike
- [] Use the iv to find out naked put horizon strikes, based on PUTSTDMULT for 30-45 dtes
- [] Find out the option price and option volatility and margin of the horizon strikes
- [] Update the price, volatility and margin if available
- [] Calculate the return on margin (expROM)
- [] Calculate the risk based on 3SD
- [] Computed expPrice based on expROM
- [] Generate 365 day history
- [] Calculate the historical volatility backwards on dte
- [] Calculate the historical volatility backwards on dte last year
- [] Calculate the hiistorical volatility backwards on equivalent dte gap since last earnings 

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_async import util
util.startLoop()

# Set the root
from from_root import from_root
from loguru import logger

ROOT = from_root()
LOGPATH = ROOT/'log'/f"one_symbol.log"

# logger.add(LOGPATH, colorize=False, backtrace=True, diagnose=True)
logger.add(LOGPATH, colorize=False)

1

In [ ]:
# Set a symbol

MARKET = 'NSE'

if MARKET.upper() == 'SNP':
    symbol = 'GD'
else:
    symbol = 'RELIANCE'

In [ ]:
# Imports 
from utils import Vars, get_pickle

_vars = Vars(MARKET)
PORT = port = _vars.PAPER
OPT_COLS = _vars.OPT_COLS[0]
COLS = _vars.COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# One symbol chain

In [ ]:
# Imports
import asyncio

from ib_async import Option

from utils import (get_exchange_name, get_margins, get_pickle,
                   get_strike_closest_to_und, get_unds, make_a_raw_contract,
                   make_chains, qualify_naked_opts, is_file_age_older_than, delete_files)

In [ ]:
# Get the underlYing contract for the symbol

UNDS_PATH = DATAPATH / 'unds.pkl'

if is_file_age_older_than(UNDS_PATH):
    delete_files(str(UNDS_PATH))

contract = get_unds(MARKET).get(symbol, None)


In [ ]:
# Get all the chains of the symbol
df_sym_chain = asyncio.run(make_chains(contracts=contract, port=port))

In [ ]:
# Get the nearest dte chains
dfn = df_sym_chain[df_sym_chain.dte == df_sym_chain.dte.min()].\
        reset_index(drop=True)

dfn = df_sym_chain # !!! TEMPORARY TO TEST MARGINS

In [ ]:
# Get the closest strikes to und for the nearest chain
# Use -2 for puts and 2 for calls to get at least 2 good options near undprice

df_c = get_strike_closest_to_und(dfn, 2)
calls_cond = (df_c.strike > df_c.undPrice) & (df_c.right == 'C')
df_c = df_c[calls_cond]

df_p = get_strike_closest_to_und(dfn, -2)
puts_cond = (df_p.strike <= df_p.undPrice) & (df_p.right == 'P')
df_p = df_p[puts_cond]

df_chains = pd.concat((df_c, df_p), ignore_index=True)

df_ch = df_chains.sort_values(['dte', 'right'], ascending=(True, True))

In [ ]:
# Get margins and commissions from the options closest to und for each dte

#... qualify
df_chq = qualify_naked_opts(df_ch)

# ... remove NA
df_mgn = df_chq.dropna(subset=['margin', 'comm'])

In [ ]:
df_mgn.head()

In [ ]:
import math
(df_mgn.undPrice*df_mgn.iv*df_mgn.dte.apply(lambda x: math.sqrt(x/365))*5.3*df_mgn.multiplier*df_mgn.lot)[:1]

In [ ]:
# ... Get the closest for each dte
g = df_mgn.groupby(['symbol', 'dte'])

margin = df_mgn.loc[g.apply(lambda x: abs(x.strike-x.undPrice).idxmin())]


In [ ]:
margin

In [ ]:
# ... get ivs of each of these closest options
from utils import get_mkt_prices
from tqdm import tqdm
from ib_async import IB

In [ ]:
opt_contract = dfn.contract.sample(3).iloc[0]
opt_contract

from utils import get_tick_data

async def main(contract):
    return await get_tick_data(ib, opt_contract, 5)

with IB().connect(port=port) as ib:
    # ticker = asyncio.run(main(opt_contract))
    ticker = ib.reqTickByTickData(contract, 'BidAsk')
    ib.sleep(10)

    print(ticker)

    ib.cancelTickByTickData(ticker.contract, 'BidAsk')

In [ ]:
ib = IB().connect(port=port)

In [ ]:
ib.__dict__

In [ ]:
from utils import to_list, chunk_me, split_symbol_price_iv, clean_ib_util_df, get_tick_data, get_market_data
import math
async def get_a_price_iv(ib, contract, sleep: float=2) -> dict:

    """[async] Computes price and IV of a contract.

    OUTPUT: dict{localsymbol, price, iv}
    
    Could take upto 12 seconds in case live prices are not available"""

    mkt_data = await get_market_data(ib, contract, sleep)

    if math.isnan(mkt_data.marketPrice()):

        if math.isnan(mkt_data.close):
            tick_data = await get_tick_data(ib, contract)
            tick_data = tick_data[0]

            if math.isnan(tick_data.marketPrice()):
                undPrice = tick_data.close
                if math.isnan(undPrice):
                    logger.info(f"No price found for {contract.localSymbol}!")
            else:
                undPrice = tick_data.marketPrice()
        else:
            undPrice = mkt_data.close

    else:
        undPrice = mkt_data.marketPrice()

    # get iv from modelGreeks, if impliedVolatility not found
    # iv = mkt_data.impliedVolatility
    # if not iv:
    iv = mkt_data

    price_iv = (contract.localSymbol, undPrice, iv)

    return price_iv


async def get_mkt_prices(port: int, 
                         contracts: list, 
                         chunk_size: int=44, 
                         sleep: int=7) -> pd.DataFrame:
    
    """[async] A faster way to get market prices.
    """

    contracts = to_list(contracts)
    chunks = tqdm(chunk_me(contracts, chunk_size), desc="Mkt prices with IVs")
    results = dict()
    with await IB().connectAsync(port=port) as ib:
        for cts in chunks:
            tasks = [asyncio.create_task(get_a_price_iv(ib, c, sleep)) for c in cts]
            res = await asyncio.gather(*tasks)

            for r in res:
                symbol, price, iv = r
                results[symbol] = (price, iv)

    df_prices = results

    # df_prices = split_symbol_price_iv(results)
    # df_prices = pd.merge(clean_ib_util_df(contracts).iloc[:, :6], df_prices, on='symbol')

    # # remove unnecessary columns (for secType == `STK`)
    # keep_cols = ~((df_prices == 0).all() | \
    #           (df_prices == "").all() | \
    #             df_prices.isnull().all())
    
    # df_prices = df_prices.loc[:, keep_cols[keep_cols == True].index]

    return df_prices

In [ ]:
df_price = asyncio.run(get_mkt_prices(port=PORT, contracts=dfn.contract.sample(3), sleep=10))

In [ ]:
df_price

In [ ]:
dfn.contract.sample(1).iloc[0]

In [ ]:
from utils import get_market_data
with IB().connect(port=port) as ib:
    mkt_data = asyncio.run(get_market_data(ib, c=dfn.contract.sample(1).iloc[0], sleep=5))

In [ ]:
mkt_data.modelGreeks.impliedVol

In [ ]:
df_price

In [ ]:
dfn.contract